# EXAM: INTELLIGENT SYSTEMS FOR PATTERN RECOGNITION
PATERNITI BARBINO NIKO - MAT:(638257)

**Coding Project N°7-Music generation**:
Implement a music generation deep neural network integrating a variational auto-encoder approach with transformers.


I used the paper https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5 as inspiration and the relative code demonstration to better understand the task of music generation
https://colab.research.google.com/drive/19TQqekOlnOSW36VCL8CPVEQKBBukmaEQ#scrollTo=DDOBVWULXfpz

The following paper has been very useful and allowed me to understand some concepts used in this project
https://colinraffel.com/publications/thesis.pdf

Importing the files from Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the Clean Lakh MIDI Dataset
https://colinraffel.com/projects/lmd/


In [ ]:
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar -xzvf "/content/clean_midi.tar.gz" -C "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset"

--2023-09-04 10:28:21--  http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
Resolving hog.ee.columbia.edu (hog.ee.columbia.edu)... 128.59.66.5
Connecting to hog.ee.columbia.edu (hog.ee.columbia.edu)|128.59.66.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 234283029 (223M) [application/x-gzip]
Saving to: ‘clean_midi.tar.gz’

clean_midi.tar.gz     8%[>                   ]  18.67M  9.33MB/s               ^C
clean_midi/
clean_midi/Hugues Aufray/
tar: clean_midi/Hugues Aufray: Cannot mkdir: Read-only file system
clean_midi/Hugues Aufray/Celine.mid
tar: clean_midi/Hugues Aufray: Cannot mkdir: Read-only file system
tar: clean_midi/Hugues Aufray/Celine.mid: Cannot open: No such file or directory
clean_midi/Linda Ronstadt/
tar: clean_midi/Linda Ronstadt: Cannot mkdir: Read-only file system
clean_midi/Linda Ronstadt/When Will I Be Loved.mid
tar: clean_midi/Linda Ronstadt: Cannot mkdir: Read-only file system
tar: clean_midi/Linda Ronstadt/When Will I Be Loved.mid

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
pip install compressive_transformer_pytorch

In [4]:
!pip install pretty_midi
!pip install plotting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=53408858bae4584cca9d744deeb6ecc55731c92f66816eee88938f7f6ba4beba
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [5]:
#Importing libraries
import torch
import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from compressive_transformer_pytorch import CompressiveTransformer
from compressive_transformer_pytorch import AutoregressiveWrapper
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset
from keras import utils
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import glob
import pickle
from music21 import converter, instrument, stream, note, chord
import math
import shutil
from collections import defaultdict

In [6]:
output_dir = '/content/drive/My Drive/ISPR_PRJ/out/best_model/'
current_path ='/content/drive/My Drive/ISPR_PRJ/'

Hyperparameters setting

In [7]:
#Training:

epochs =  100
batch_size = 64
learning_rate=1e-3

#Vector-space embedding:
n_dim = 64
sequence_length = 64

VALIDATE_EVERY  = 5
GENERATE_EVERY  = 500


In [8]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, output_dir+filename)
    if is_best:
        shutil.copyfile(output_dir+filename, output_dir+'best.pth.tar')

In [9]:
!nvidia-smi

Tue Sep  5 05:50:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Statistics

To better understand the dataset i used the following link

https://nbviewer.org/github/craffel/midi-ground-truth/blob/master/Statistics.ipynb

In [10]:
import pretty_midi
import numpy as np
import joblib
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
import matplotlib.gridspec
import collections
import os
import plotting

In [11]:
def compute_statistics(midi_file):
    """
    Given a path to a MIDI file, compute a dictionary of statistics about it

    Parameters
    ----------
    midi_file : str
        Path to a MIDI file.

    Returns
    -------
    statistics : dict
        Dictionary reporting the values for different events in the file.
    """
    # Some MIDI files will raise Exceptions on loading, if they are invalid.
    # We just skip those.
    try:
        pm = pretty_midi.PrettyMIDI(midi_file)
        # Extract informative events from the MIDI file
        return {'n_instruments': len(pm.instruments),
                'program_numbers': [i.program for i in pm.instruments if not i.is_drum],
                'key_numbers': [k.key_number for k in pm.key_signature_changes],
                'tempos': list(pm.get_tempo_changes()[1]),
                'time_signature_changes': pm.time_signature_changes,
                'end_time': pm.get_end_time(),
                'lyrics': [l.text for l in pm.lyrics]
                }
    # Silently ignore exceptions for a clean presentation (sorry Python!)
    except Exception as e:
        pass

In [12]:
data_path = current_path+"midi_songs/dataset/clean_midi"
compute_statistics(current_path+"midi_songs/dataset/Genres/bluesRock/The Rolling Stones/19th Nervous Breakdown.mid")

{'n_instruments': 9,
 'program_numbers': [29, 32, 30, 28, 25, 81, 33, 27],
 'key_numbers': [0],
 'tempos': [190.0002850004275],
 'time_signature_changes': [TimeSignature(numerator=4, denominator=4, time=0.0)],
 'end_time': 238.775957625,
 'lyrics': []}

# Data Processing

As we can see music21 represents music.
We have, two different kind of elements:
* Notes
* Chords

We also have the time offset of each element(this is the time when the note or chord must be played).

In [13]:
file = current_path+"midi_songs/dataset/clean_midi/Jackson 5/I Want You Back.mid"
midi = converter.parse(file)
notes_to_parse = midi.flatten().notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.note.Unpitched object at 0x7af903b831c0> 4.0
<music21.note.Unpitched object at 0x7af903b83280> 5.0
<music21.note.Unpitched object at 0x7af903b83340> 6.0
<music21.chord.Chord C7 B6 A6 G6 F6 E6> 7.0
<music21.chord.Chord D6 C6 B5 A5 G5 F5> 7.25
<music21.chord.Chord E5 D5 C5 B4 A4 G4> 7.5
<music21.chord.Chord F4 E4 D4 C4 B3 A3> 7.75
<music21.note.Note G#> 8.0
<music21.note.Note G#> 8.0
<music21.chord.Chord G#4 G#3> 8.0


#Artists for each genre

In [14]:
import os
import shutil

PATH = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/lakh_playlist/playlists"
source = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/clean_midi"

for y in sorted(os.listdir(PATH)):
  z = open(PATH+"/"+y)
  name = z.read()

  temp = os.path.splitext(y)
  genre = temp[0]

  print("GENERE: "+genre+"\n")
  print(name)

  z.close()

GENERE: alternativeRock

Radiohead
Alice in Chains
Beck
Bush
Depeche Mode
Foo Fighters
Hole
Jane's Addiction
Nirvana
Pearl Jam
Sonic Youth
The Smashing Pumpkins
The Smiths
Weezer
4 Non Blondes

GENERE: artRock

Pink Floyd
The Who
Queen
Zappa
Zappa Frank
King Crimson
Camel
Roxy Music
Yes
Genesis
Tool
Peter Gabriel
10cc
Tool


GENERE: blues

B.B. King
Duke Robillard
Jimmy Reed
John Lee Hooker
Johnny Shines
Professor Longhair
Rufus Thomas
Wilbert Harrison

GENERE: bluesRock

Guess Who
Bachman-Turner Overdrive
Santana
ZZ Top
Led Zeppelin
The Allman Brothers Band
The Rolling Stones
The Yardbirds
Canned Heat
Ten Years After
Fleetwood Mac
Jeff Beck
The Animals
The Doors
AC DC
Lynyrd Skynyrd
Aerosmith
Derek and the Dominos
Bad Company
Status Quo
Pink Floyd
Cream
Eric Clapton
Jeff Beck
Deep Purple
Black Sabbath
Three Dog Night
Grand Funk Railroad


GENERE: country

Alabama
Alan Jackson
Anne Murray
Bobby Bare
Buck Owens
Charlie Rich
Conway Twitty
Crystal Gayle
Dixie Chicks
Eddy Arnold
Floyd Cram

In [ ]:
'''
dest = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/Genres/country"

midi_files = os.listdir(source)
for f in sorted(midi_files):
  if f == "Willie Nelson":
    shutil.move(source+"/"+f,dest)
    print("done")
'''

done


Function to estract the name of a file(the song's name)
and the name of the group



In [15]:
def get_file_name(file_path):
	file_path_components = file_path.split('/')
	file_name_and_extension = file_path_components[-1].rsplit('.', 1)
	return file_name_and_extension[0]

def get_group_name(path):
  for x in os.listdir(path):
    name = x
    return name


In [16]:
get_file_name("/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/Genres/alternativeRock/Foo Fighters/Everlong.mid")

'Everlong'

For this project i decided to use alternativeRock,artRock and bluesRock songs all having the acoustic guitar as instrument, identified by the program '25'.

In [17]:
def compute_statistics(midi_file):
    try:
        pm = pretty_midi.PrettyMIDI(midi_file)
        name_path = midi_file
        result = get_file_name(name_path)
        for i in pm.instruments:
          if i.program == 25:
            return {'Song': result,
                    'n_instruments': len(pm.instruments),
                    'program_numbers':[i.program for i in pm.instruments if not i.is_drum],
                   }
    except Exception as e:
        pass

Here we compute in parallel the statistics of the songs for each genre


In [ ]:
Genres_path = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/Genres"
statistics = joblib.Parallel(n_jobs=10, verbose=0)(
    joblib.delayed(compute_statistics)(midi_file)
    for midi_file in glob.glob(os.path.join(Genres_path, 'bluesRock', '*', '*.mid')))

statistics = [s for s in statistics if s is not None]

In [20]:
import pandas as pd
df = pd.DataFrame(statistics)
df

NameError: ignored

Now i sample 120 files which will be the ones i will use for the experiments

In [ ]:
df2=df.sample(n=120)
df2

,Song,n_instruments,program_numbers
216,You Ain't Seen Nothin Yet.2,10,"[25, 35, 27, 75, 27, 30, 30, 30, 30]"
12,Jumpin' Jack Flash.1,9,"[35, 25, 29, 28, 27, 27, 22, 2]"
114,Your Time Is Gonna Come,10,"[18, 33, 25, 22, 52, 25, 25, 26, 116]"
89,The Battle Of Evermore.2,6,"[73, 72, 24, 24, 24, 25]"
208,If You Needed Somebody.1,5,"[100, 25, 25, 1]"
...,...,...,...
126,Laughing,12,"[26, 27, 63, 35, 25, 1, 18, 52, 52, 4, 27]"
160,She's Waiting,15,"[27, 30, 29, 81, 35, 24, 24, 25, 17, 90, 29, 2..."
167,Sisters Of The Moon,11,"[75, 33, 5, 26, 25, 30, 29, 28, 31, 53]"
178,Gypsy.2,11,"[28, 33, 25, 27, 51, 24, 52, 73, 29, 8]"


In [ ]:
df2.loc[:,"Song"]

216    You Ain't Seen Nothin Yet.2
12            Jumpin' Jack Flash.1
114        Your Time Is Gonna Come
89        The Battle Of Evermore.2
208       If You Needed Somebody.1
                  ...             
126                       Laughing
160                  She's Waiting
167            Sisters Of The Moon
178                        Gypsy.2
65                   Maria Maria.1
Name: Song, Length: 120, dtype: object

In [ ]:
file_path = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/Genres/bluesRock"
col_list = df2.Song.values.tolist()

for element in col_list:
  for midi_file in glob.glob(os.path.join(file_path, '*', '*.mid')):
    midi_song = get_file_name(midi_file)
    if element == midi_song:
      print(midi_song)
      shutil.copy(midi_file,"/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/train/120guitarbluesRock")




You Ain't Seen Nothin Yet.2
Jumpin' Jack Flash.1
Your Time Is Gonna Come
The Battle Of Evermore.2
If You Needed Somebody.1
Tears in Heaven
Little Lies
No Time.2
Beast of Burden
Let It Grow.4
Ruby Tuesday.2
Let It Grow.1
Let It Grow.2
Gimme Shelter
Going to California.1
You Can't Always Get What You Want.1
Stairway to Heaven.8
Light My Fire.10
Stairway To Heaven.11
That's the Way
Silver Springs.1
Change the World.2
Little Lies.3
Dreams.5
Sara.4
Gold Dust Woman
Wild Horses
Melissa
San Franciscan Nights
Angie.7
Tears in Heaven.3
Tusk
Paint It Black.1
The Game of Love
Black Country Woman
Sentimental Lady
White Room.1
White Room.1
Layla
Layla
Hand Me Down World
Samba Pa Ti
The Battle Of Evermore
Bell Bottom Blues
Midnight Rider
Dreams.4
These Eyes.1
Sky Pilot
All My Love
Ozone Baby
Ten Years Gone
What It Takes
Let It Grow
Landslide.1
Affairs of the Heart.2
Run With The Pack.1
Brown Sugar.2
Honky Tonk Woman.2
No Sugar Tonight.1
Change the World.1
Ten Years Gone.1
Stairway to Heaven.6
Janie's

In [76]:
source = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/Genres/alternativeRock/Janes Addiction/Stop.mid"
dest = "/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/small_dataset/test"
shutil.copy(source,dest)

'/content/drive/MyDrive/ISPR_PRJ/midi_songs/dataset/small_dataset/test/Stop.mid'

Here we process all MIDI files, obtaining data from each note of chord.
* If I process a note, i will store in the list a string representing the pitch (the note name) and the octave.
* If I process a chord (Chords are set of notes played at the same time) i will store a different type of string with numbers separated by dots.

Each number represents the pitch of a chord note.
So i am creating a big list with all the elements of all the compositions

In [82]:
notes_for_instruments = []
for i,file in enumerate(glob.glob(current_path+"midi_songs/dataset/small_dataset/train/*.mid")):
      midi = converter.parse(file)
      print('Parsing file ', i, " ", file)
      notes_to_parse = None
      try:  # file has instrument parts
          s2 = instrument.partitionByInstrument(midi)
          notes_to_parse = s2.recurse()
      except:  # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      notes_instrument = []
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes_instrument.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes_instrument.append('.'.join(str(n) for n in element.normalOrder))
      notes_for_instruments.append(notes_instrument)
with open(current_path + 'notes_for_instruments', 'wb') as filepath:
    pickle.dump(notes_for_instruments, filepath)



Parsing file  0   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/About a Girl.3.mid
Parsing file  1   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Big Me.mid
Parsing file  2   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Comedown.mid


/usr/local/lib/python3.10/dist-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


Parsing file  3   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Everlong.1.mid
Parsing file  4   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/In Your Room.mid
Parsing file  5   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Love.mid
Parsing file  6   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Nutshell.mid
Parsing file  7   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Today.4.mid
Parsing file  8   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Loser.mid
Parsing file  9   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Violet.mid
Parsing file  10   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Abacab.2.mid
Parsing file  11   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/train/Doing All Right.mid
Parsing file  12   /content/drive/My Drive/ISPR_PRJ/midi_songs/datas

In [19]:

notes_for_instruments_validation = []
for i,file in enumerate(glob.glob(current_path+"midi_songs/dataset/small_dataset/validation/*.mid")):
      midi = converter.parse(file)
      print('Parsing file ', i, " ", file)
      notes_to_parse = None
      try:  # file has instrument parts
          s2 = instrument.partitionByInstrument(midi)
          notes_to_parse = s2.recurse()
      except:  # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      notes_instrument = []
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes_instrument.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes_instrument.append('.'.join(str(n) for n in element.normalOrder))
      notes_for_instruments_validation.append(notes_instrument)
with open(current_path + 'VALIDATION_notes_for_instruments', 'wb') as filepath:
    pickle.dump(notes_for_instruments_validation, filepath)


Parsing file  0   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Knockin' on Heaven's Door.mid
Parsing file  1   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Hey You.mid
Parsing file  2   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Going Up the Country.mid
Parsing file  3   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Fake Plastic Trees.mid
Parsing file  4   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Bullet Proof..I Wish I Was.mid
Parsing file  5   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Our Love Is the Sea.mid
Parsing file  6   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/It's a Hard Life.2.mid
Parsing file  7   /content/drive/My Drive/ISPR_PRJ/midi_songs/dataset/small_dataset/validation/Innuendo.1.mid
Parsing file  8   /content/drive/My Drive/ISPR_PRJ/midi_s

If the midi file processing has already been done before, we can load the data directly


In [20]:
with open(current_path + 'notes_for_instruments', 'rb') as f:
    notes_for_instruments = pickle.load(f)
with open(current_path + 'VALIDATION_notes_for_instruments', 'rb') as f:
    notes_for_instruments_validation = pickle.load(f)

I obtain the number of different notes in our dataset, because this will be the number of
possible output classes of our model

In [21]:
#N° of possible outputs
print("Possible outputs: ")
print(len(set(item for notes_for_instrument in notes_for_instruments for item in notes_for_instrument)))

#N° possible outputs validation
print("Possible outputs (validation)")
print(len(set(item for notes_for_instrument in notes_for_instruments_validation for item in notes_for_instrument)))

Possible outputs: 
326
Possible outputs (validation)
275


To prepare our data we:

*   Map each pitch or chord to an integer

*   Create pairs of input sequences and its corresponding output note

*   The network will make its prediction of the next
note (or chord), based on the previous sequence_length notes (or chords).


In [22]:
#Get pitch names
pitchnames_training = set(item for notes_for_instrument in notes_for_instruments for item in notes_for_instrument)
pitchnames_validation = set(item for notes_for_instrument in notes_for_instruments_validation for item in notes_for_instrument)
pitchnames = sorted(pitchnames_training.union(pitchnames_validation))

In [23]:
n_vocab = len(pitchnames)
n_vocab

406

Map pitches to integers


In [24]:
note_to_int = defaultdict(int)
note_to_int.update(dict((note, number) for number, note in enumerate(pitchnames)))
network_input = []

for notes in notes_for_instruments:
    if len(notes) - sequence_length<=0:
        print("canzone troppo corta")

    #Create input sequences and the corresponding outputs
    for j in range(0, len(notes) - sequence_length, sequence_length//2):
      for i in range(j, len(notes) - sequence_length, sequence_length):
        # Map pitches of sequence_in to integers
        network_input.append([note_to_int[char] for char in notes[i:i + sequence_length]])

n_patterns = len(network_input)
#Reshape the input into a format compatible with Transormer layers
network_input = np.reshape(network_input, (n_patterns, sequence_length))

In [25]:
network_input

array([[ 97,  97, 124, ..., 124,  97,  97],
       [ 97, 124, 124, ..., 124, 124, 124],
       [124, 124,  97, ..., 366, 395, 366],
       ...,
       [352, 352, 352, ...,  22, 320,  22],
       [352, 352, 352, ..., 352, 352, 352],
       [352, 352, 352, ...,  22, 320,  22]])

In [26]:
note_to_int_validation = defaultdict(int)
note_to_int_validation.update(dict((notes_validation, number) for number, notes_validation in enumerate(pitchnames)))
network_input_validation = []
network_output_validation = []
for notes_validation in notes_for_instruments_validation:
    if len(notes_validation) - sequence_length<=0:
        print("canzone troppo corta")
    # create input sequences and the corresponding outputs
    for j in range(0, len(notes_validation) - sequence_length, sequence_length//2):
      for i in range(j, len(notes_validation) - sequence_length, sequence_length):
        # Map pitches of sequence_in to integers
        network_input_validation.append([note_to_int_validation[char] for char in notes_validation[i:i + sequence_length]])
n_patterns_validation = len(network_input_validation)
# reshape the input into a format compatible with Transormer layers
network_input_validation = np.reshape(network_input_validation, (n_patterns_validation, sequence_length))

In [27]:
network_input_validation

array([[404, 104, 353, ..., 374, 179, 353],
       [375, 104, 353, ..., 359, 403, 352],
       [374,  22, 235, ..., 321, 121, 235],
       ...,
       [126,   0,  72, ..., 192, 148, 126],
       [244, 234,  88, ..., 283,   0, 210],
       [126,   0,  72, ..., 192, 148, 126]])

In [28]:
print(network_input.shape)
print(network_input_validation.shape)

(39231, 64)
(15636, 64)


# Neural Network structure

Pytorch implementation of Compressive Transformers is a variant of Transformer-XL with compressed memory for long-range language modelling.

When training, we used the AutoregressiveWrapper to take care of memory management across segments.

https://github.com/lucidrains/compressive-transformer-pytorch

In [29]:
def create_network(sequence_length, n_vocab):
    model = CompressiveTransformer(
    num_tokens = n_vocab,
    dim = sequence_length,
    depth = 6,
    seq_len = sequence_length,
    mem_len = sequence_length,
    cmem_len = 256,
    cmem_ratio = 4,
    memory_layers = [5,6],
    gru_gated_residual = False,
    #post-attention dropout
    attn_dropout = 0.1,
    #feedforward dropout
    ff_dropout = 0.1,
    #attention layer output dropout
    attn_layer_dropout = 0.1
    )

    model = AutoregressiveWrapper(model)
    model.cuda()
    return model


Model Summary

In [33]:
model = create_network(sequence_length,n_vocab)
print(model)

AutoregressiveWrapper(
  (net): CompressiveTransformer(
    (token_emb): Embedding(406, 64)
    (to_model_dim): Identity()
    (to_logits): Sequential(
      (0): Identity()
      (1): Linear(in_features=64, out_features=406, bias=True)
    )
    (attn_layers): ModuleList(
      (0-5): 6 x Residual(
        (fn): PreNorm(
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (fn): SelfAttention(
            (compress_mem_fn): ConvCompress(
              (conv): Conv1d(64, 64, kernel_size=(4,), stride=(4,))
            )
            (to_q): Linear(in_features=64, out_features=64, bias=False)
            (to_kv): Linear(in_features=64, out_features=128, bias=False)
            (to_out): Linear(in_features=64, out_features=64, bias=True)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (dropout): Dropout(p=0.1, inplace=False)
            (reconstruction_attn_dropout): Dropout(p=0.0, inplace=False)
          )
        )
      )
    )
    (ff_

In [34]:
def cycle(loader):
    while True:
        for data in loader:
          yield data

data_train = torch.from_numpy(network_input).cuda()
train_loader = torch.utils.data.DataLoader(data_train, batch_size=32)
cycle_train_loader  = cycle(DataLoader(data_train, batch_size = data_train.shape[0]))

#N° of batches
num_batches=math.ceil(data_train.shape[0]/batch_size)
print(num_batches)

data_validation = torch.from_numpy(network_input_validation).cuda()
validation_loader = torch.utils.data.DataLoader(data_validation, batch_size=32)
cycle_validation_loader  = cycle(DataLoader(data_validation, batch_size = data_validation.shape[0]))

#N° of batches
num_batches_val=math.ceil(data_validation.shape[0]/batch_size)
print(num_batches_val)

613
245


Adam Pytorch Optimizer

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training

In [58]:
trainingEpoch_loss = []
validationEpoch_loss = []

for i in tqdm.tqdm(range(0,epochs), mininterval=20., desc='training'):
    train_step_loss = []
    val_step_loss = []
    model.train()
    tot_loss = 0.0
    is_best=0
    best_loss_value=n_vocab
    avg_loss_val=0

    for mlm_loss, aux_loss, is_last in model(next(cycle_train_loader), max_batch_size = batch_size, return_loss = True):
        loss = mlm_loss + aux_loss

        loss.backward()

        tot_loss+=loss;

        if is_last:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            optimizer.zero_grad()

    if i % VALIDATE_EVERY == 0 or i==epochs-1:
      model.eval()
      with torch.no_grad():
          for loss_val, aux_loss_val, is_last_val in model(next(cycle_validation_loader),max_batch_size = batch_size, return_loss = True):
            avg_loss_val+=loss_val/num_batches_val;

            validationEpoch_loss.append(avg_loss_val.item())

            if is_last_val:
              print(f'\n validation loss: {avg_loss_val.item():.4f}')

          #validationEpoch_loss.append(val_step_loss)



    avg_loss=tot_loss/num_batches

    if i%5==0 or i==epochs-1:
      if best_loss_value>avg_loss:
        best_loss_value=avg_loss;
        is_best=1

      save_checkpoint({
      'epoch': i,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict' : optimizer.state_dict(),
      'loss':avg_loss.item(),
     }, is_best, 'best_model_64'+str(i)+'_'+"{:.4f}".format(avg_loss.item())+'.pth.tar')
      is_best=0

     #train_step_loss.append(avg_loss.item())
    trainingEpoch_loss.append(avg_loss.item())



    print(f'\n Epoch: {i} |Training loss: {avg_loss.item():.4f}')
print('\nTraining complete.')



training:   1%|          | 1/100 [00:27<45:04, 27.32s/it]


 validation loss: 6.0241

 Epoch: 0 |Training loss: 0.8095


training:   2%|▏         | 2/100 [00:52<42:16, 25.88s/it]


 Epoch: 1 |Training loss: 0.8072


training:   3%|▎         | 3/100 [01:17<41:24, 25.61s/it]

...


 Epoch: 97 |Training loss: 0.6121


training:  99%|█████████▉| 99/100 [42:27<00:25, 25.41s/it]


 Epoch: 98 |Training loss: 0.6106


training: 100%|██████████| 100/100 [42:56<00:00, 25.76s/it]


 validation loss: 6.6550

 Epoch: 99 |Training loss: 0.6090

Training complete.


In [60]:
weights = "best.pth.tar"
checkpoint = torch.load(output_dir+weights)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [53]:
note_to_int = defaultdict(int)
note_to_int.update(dict((note, number) for number, note in enumerate(pitchnames)))
network_input = []

# Generate network input again
network_input_generation = []
for notes in notes_for_instruments:
    if len(notes) - sequence_length<=0:
        print("canzone troppo corta")
    for i in range(0, len(notes) - sequence_length, 1):
      network_input_generation.append([note_to_int[char] for char in notes[i:i + sequence_length]])
n_patterns_generation = len(network_input_generation)
network_input_generation = np.reshape(network_input_generation, (n_patterns_generation, sequence_length))

In [54]:
network_input_generation.shape


(61775, 64)

The workflow now is:

* Pick a seed sequence randomly from your list of inputs (pattern variable)
* Pass it as input for your model to generate a new element (note or chord)
* Add the new element to your final song and to your pattern list
* Remove the first item from pattern
* Go to step 2

So we generate notes from the neural network based on a sequence of notes picking a random sequence from the input as a starting point for the prediction

In [55]:
start = np.random.randint(0, len(network_input_generation)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = torch.from_numpy(network_input_generation[start]).cuda()

prediction_output = model.generate(pattern, 500)

In [56]:
result_sample=[]

for i in range(500):
  print(i)
  result = int_to_note[prediction_output[i].item()]
  print('\r', 'Predicted ', i, " ",result, end='')
  result_sample.append(result)

prediction_output=result_sample

0
 Predicted  0   G#21
 Predicted  1   G#22
 Predicted  2   G#23
 Predicted  3   G#34
 Predicted  4   B45
 Predicted  5   B46
 Predicted  6   G#27
 Predicted  7   G#38
 Predicted  8   E-59
 Predicted  9   G#210
 Predicted  10   G#311
 Predicted  11   G#412
 Predicted  12   G#213
 Predicted  13   G#314
 Predicted  14   G#415
 Predicted  15   G#216
 Predicted  16   G#317
 Predicted  17   B418
 Predicted  18   G#219
 Predicted  19   G#320
 Predicted  20   G#421
 Predicted  21   G#222
 Predicted  22   G#323
 Predicted  23   G#224
 Predicted  24   G#325
 Predicted  25   G#426
 Predicted  26   G#227
 Predicted  27   G#328
 Predicted  28   G#429
 Predicted  29   G#230
 Predicted  30   G#331
 Predicted  31   B432
 Predicted  32   B433
 Predicted  33   F#434
 Predicted  34   G#235
 Predicted  35   G#336
 Predicted  36   E-537
 Predicted  37   G#238
 Predicted  38   G#339
 Predicted  39   G#240
 Predicted  40   G#341
 Predicted  41   G#442
 Predicted  42   G#243
 Predicted  43   G#344
 Predicted

The last step is creating a MIDI file from the predictions using music21

We create a Stream and add to it the predicted notes and chords adding an offset of 0.5 between elements.

In [57]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.AcousticGuitar()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.AcousticGuitar()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='MySong.mid')

'MySong.mid'